# Working with Spotify playlists
Getting a playlist from a Spotify object and running basic operations with the playlist.

## Import statements

In [ ]:
# import os
#
# from pathlib import Path
# from dotenv import load_dotenv
#
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
#
# # Based on https://github.com/ipython/ipynb; the reported compiler errors don't seem to matter
# import ipynb
# # from ipynb.fs.full.spotify_authentication import get_spotify_object
#
from my_spotify import get_spotify_object

# from my_spotify import get_spotify_object
# import my_spotify
# %run "M:\\Vladan\\Courses\\P3\\Python\\Python projects\\Jupyter projects\\spotipy-getting-started\\spotify_authentication.ipynb"

## Define playlists to work with
A Spotify playlist can be identified by its ID, its URI, and its URL. See the differences [here](https://spotipy.readthedocs.io/en/2.22.1/#ids-uris-and-urls). Spotify URLs are used in this notebook. See also [this](https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50), section *Extracting Tracks From a Playlist*.

To get any of these playlist identifiers, go to Spotify, open a playlist of your choice, click the '.&nbsp;. .' under the playlist title, and then select *Share > Copy link to playlist*.

In [ ]:
THE_BEATLES_PLAYLIST_URL_1 = 'https://open.spotify.com/playlist/6ZmqDRJKJf3v3LzYZAaGGU?si=6b487ba5f9b54374' # 215 songs
THE_BEATLES_PLAYLIST_URL_2 = 'https://open.spotify.com/playlist/01pA1OnfpqfS8Q4MeIbBef?si=03d0ad93c5a04c9c' # 219 songs
THE_BEATLES_PLAYLIST_URL_3 = 'https://open.spotify.com/playlist/0eb2h4Qy3n43wLggYf5qEo?si=3ab4e29b207e4892' # 216 songs
THE_BEATLES_PLAYLIST_URL_4 = 'https://open.spotify.com/playlist/4dRYcGMAbZWrrDxfeBU7IB?si=62d59e6a73d944e4' # 237 songs
THE_BEATLES_PLAYLIST_URL_5 = 'https://open.spotify.com/playlist/6Z2fd7oueYPdy5dUo34LQF?si=981aaa474b01497e' # 213 songs
THE_BEATLES_PLAYLIST_URL_6 = 'https://open.spotify.com/playlist/24Wec2tWqPZKEUkh0V6lXz?si=27bb6f162d214322' # 226 songs
THE_BEATLES_PLAYLIST_URL_7 = 'https://open.spotify.com/playlist/2nr5EYhGrWFlOzmw9El6dF?si=27d2fb443d3141b0' # 215+145 songs (solo)

## Get the Spotify object

In [ ]:
# display(get_spotify_object('env/.env'))
spot = get_spotify_object('env/.env')

## Get a playlist

In [ ]:
pl1 = spot.playlist(THE_BEATLES_PLAYLIST_URL_1)

## Check what some items of the playlist dictionary are

In [ ]:
# print(pl1)
# type(pl1)                         # dict

# for k, v in pl1.items():
#     print(str(k) + ': ' + str(v))

display(pl1.keys())
display(pl1['id'])
display(pl1['uri'])
display(pl1['snapshot_id'])
display(pl1['type'])
display(pl1['collaborative'])
display(pl1['external_urls'])
display(pl1['followers'])
display(pl1['href'])
display(pl1['images'])
display(pl1['name'])

## Get the tracks from the playlist
<u>**Important:**</u>&nbsp;The `playlist_tracks()` method of the Spotify object will return a dictionary with only the first 100 tracks max under its `items` key. If there are more than 100 tracks in the playlist, the `next` key in the returned dictionary will point to the next 100 tracks, and so on.

In [ ]:
tracks1 = spot.playlist_tracks(THE_BEATLES_PLAYLIST_URL_1)

## Check what some items of the tracks dictionary are

In [ ]:
# type(tracks1)                     # dict

display(tracks1.keys())
display(tracks1['total'])           # total number of tracks in the playlist
display(tracks1['href'])            # notice offset=0, limit=100
display(tracks1['limit'])           # 100
display(tracks1['offset'])          # 0
display(tracks1['next'])            # notice offset=100, limit=100
display(tracks1['previous'])        # None (this is the FIRST 100 tracks, no previous tracks)

display(tracks1['items'])           # only max 100 first tracks (if the playlist is longer than 100), not all, but very detailed
display(type(tracks1['items']))     # list
display(len(tracks1['items']))      # 100 (only the first 100 songs from the playlist)
display(tracks1['items'][0])        # the first track, very detailed
display(type(tracks1['items'][0]))  # dict

## Create the list of ALL tracks from the playlist
Remember that `<tracks>['items']` returns only the first 100 (or less, if you set `limit` to a value smaller than 100).

In [ ]:
offset = 0
all_tracks = []
tracks = spot.playlist_tracks(THE_BEATLES_PLAYLIST_URL_1, offset=offset)                        # the first 100 tracks (offset = 0)
if tracks['items']:                                                                             # if the playlist is not empty
    all_tracks.extend(tracks['items'])
    while tracks['next']:
        offset += 100
        tracks = spot.playlist_tracks(THE_BEATLES_PLAYLIST_URL_1, offset=offset)                # the next 100 tracks
        all_tracks.extend(tracks['items'])

In [ ]:
display(len(all_tracks))
display(all_tracks)

## Bundle it all together in the *get_all_tracks_from_playlist()* function
Assumption: the *.env* file is already created as explained in *spotify_authentication.ipynb*, and its relative path is passed as an argument.

In [ ]:
def get_all_tracks_from_playlist(playlist_id: str, env_file_path: str) -> list:
    spot = get_spotify_object(env_file_path)
    offset = 0
    all_tracks = []
    tracks = spot.playlist_items(playlist_id, offset=offset)           # the first 100 tracks (offset = 0)
    if not tracks['items']:                                             # if the playlist is empty
        return []
    all_tracks.extend(tracks['items'])
    while tracks['next']:
        offset += 100
        tracks = spot.playlist_items(playlist_id, offset=offset)       # the next 100 tracks
        all_tracks.extend(tracks['items'])
    return all_tracks

In [ ]:
# Test get_all_tracks_from_playlist()
all_tracks = get_all_tracks_from_playlist(THE_BEATLES_PLAYLIST_URL_1, 'env/.env')

In [ ]:
display(all_tracks)